In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !pip install datasets
# !pip install evaluate

In [2]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch
import pandas as pd
import numpy as np

In [3]:
# ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True, streaming=True)
# seq1id = 'document'
# seq2id = 'summary'

# ds = load_dataset("knkarthick/dialogsum", streaming=True)
# seq1id = 'dialogue'
# seq2id = 'summary'

ds = load_dataset("har1/MTS_Dialogue-Clinical_Note", streaming=False)
seq1id = 'dialogue'
seq2id = 'section_text'

In [4]:
ds['train']

Dataset({
    features: ['ID', 'section_header', 'section_text', 'dialogue'],
    num_rows: 1301
})

In [4]:
# sample = ds['train'][0]
# sample = next(iter(ds['train']))
ds_iter = iter(ds['train'])

In [5]:
sample = next(ds_iter)
print(sample.keys())
print('')
print(sample[seq1id])
print('')
print(sample[seq2id])

dict_keys(['ID', 'section_header', 'section_text', 'dialogue'])

Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  
Patient: No. 
Doctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?
Patient: No.  
Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?
Patient: I am seventy six years old and identify as a white female.

Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.
Diagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones
History of Patient: 76-year-ol

In [ ]:
# summarizer = pipeline('summarization', model="sshleifer/distilbart-cnn-12-6") # Bad
# summarizer = pipeline('summarization', model="sshleifer/distilbart-xsum-12-1") # Bad

# summarizer = pipeline('summarization', model='facebook/bart-large-cnn') # Good
# summarizer = pipeline('summarization', model='google/pegasus-large') # Okay
# summarizer = pipeline('summarization', model = 'facebook/bart-large-xsum') # Okay
# summarizer = pipeline('summarization', model="google/pegasus-xsum") # Good

In [6]:
device = 0 if torch.cuda.is_available() else -1

In [ ]:
# on xsum dataset: good, okay, okay, good
# on dialog dataset: okay, bad, bad, bad
# summarizers = ['facebook/bart-large-cnn', 'google/pegasus-large', 'facebook/bart-large-xsum', 'google/pegasus-xsum']

# on dialogsum dataset: good, not good
# summarizers = ['dtruong46me/train-bart-base', 'gauravkoradiya/T5-Finetuned-Summarization-DialogueDataset']
# summarizers = ['Ketan3101/ConvoBrief'] # can't run
# summarizers = ['akhil033/Flan-T5-Conversation-Summarizer'] # can't run
summarizers = ['dtruong46me/train-bart-base']

print('Ground truth: ' + sample[seq2id] + '\n')

for s in summarizers:
    summarizer = pipeline('summarization', model=s, device=device)
    summary = summarizer(sample[seq1id])
    print(s + ': ' + summary[0]['summary_text'] + '\n')
    del summarizer

In [ ]:
# summarizer = pipeline('summarization', model='dtruong46me/train-bart-base', device=device)
summarizer = pipeline('summarization', model='facebook/bart-large-cnn', device=device)

In [ ]:
sample = next(ds_iter)
print(sample[seq1id] + '\n')
print('Ground truth: ' + sample[seq2id] + '\n')
summary = summarizer(sample[seq1id])
print('Prediction: ' + summary[0]['summary_text'] + '\n')

# Clinical notes summarization

In [4]:
ds_df = pd.DataFrame(ds['train'])
ds_df.head()

,ID,section_header,section_text,dialogue
0,0,GENHX,"Symptoms: no fever, no chills, no cough, no co...",Doctor: What brings you back into the clinic t...
1,1,GENHX,"Symptoms: sudden onset headache, blurry vision...",Doctor: How're you feeling today? \nPatient: ...
2,2,GENHX,Symptoms: itching.\nDiagnosis: condylomas.\nHi...,"Doctor: Hello, miss. What is the reason for yo..."
3,3,MEDICATIONS,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: Are you taking any over the counter me...
4,4,CC,"Symptoms: Burn, right arm.\r\nDiagnosis: N/A.\...","Doctor: Hi, how are you? \nPatient: I burned m..."


In [5]:
ds_df['section_header'].value_counts()

section_header
FAM/SOCHX        373
GENHX            302
PASTMEDICALHX    122
CC                81
PASTSURGICAL      71
ROS               71
ALLERGY           64
MEDICATIONS       61
ASSESSMENT        38
EXAM              24
DIAGNOSIS         20
DISPOSITION       17
PLAN              14
EDCOURSE          11
IMMUNIZATIONS      9
IMAGING            7
GYNHX              6
PROCEDURES         4
OTHER_HISTORY      3
LABS               3
Name: count, dtype: int64

In [6]:
ds_df[ds_df['section_header'] == 'FAM/SOCHX']
# ds_df.iloc[88]
# ds['train'][41]

,ID,section_header,section_text,dialogue
8,8,FAM/SOCHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,"Doctor: Hi there, sir! How are you today? \nPa..."
9,9,FAM/SOCHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: Can you tell me about any diseases tha...
29,29,FAM/SOCHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: You had mentioned that cancer runs in ...
31,31,FAM/SOCHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: I understand that you are new to the c...
35,35,FAM/SOCHX,Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory o...,Doctor: Do you use tobacco products? Cigarette...
...,...,...,...,...
1277,76,FAM/SOCHX,Symptoms: N/A\nDiagnosis: N/A\nHistory of Comp...,Doctor: When were you diagnosed with type two ...
1278,77,FAM/SOCHX,Symptoms: N/A\nDiagnosis: N/A\nHistory of Comp...,Doctor: Can you tell me some illness that migh...
1279,78,FAM/SOCHX,Symptoms: N/A\nDiagnosis: N/A\nHistory of Comp...,Doctor: Do you drink or smoke? Or take any oth...
1286,85,FAM/SOCHX,Symptoms: N/A\nDiagnosis: N/A\nHistory of Comp...,Doctor: Do you know about any medical problems...


In [6]:
sample

{'ID': 8,
 'section_header': 'FAM/SOCHX',
 'section_text': 'Symptoms: N/A.\r\nDiagnosis: N/A.\r\nHistory of Patient: Mother died of complications from heart disease, father died of heart disease in his 40s. Two living brothers, one with unknown medical history, the other apparently healthy. One healthy child. Maternal uncles had polio, one wore crutches due to severe leg deformans, the other had leg deformities in one leg. No other family members with neurological conditions known.\r\nPlan of Action: N/A.\r\n',
 'dialogue': "Doctor: Hi there, sir! How are you today? \nPatient: Hello! I am good.\nDoctor: I would like to start with your family medical history today. What do you know about their medical history? \nPatient: My mother and father both had heart disease. Well, my mother had complication from her heart disease and that is how she passed.  My father was only in his forty's when he died. \nDoctor: I am so sorry the hear that. \nPatient: Thank you. I have two brothers, one whom I

In [5]:
sample = ds['train'][8]
print(sample.keys())
print('')
print(sample[seq1id])
print('')
print(sample[seq2id])

dict_keys(['ID', 'section_header', 'section_text', 'dialogue'])

Doctor: Hi there, sir! How are you today? 
Patient: Hello! I am good.
Doctor: I would like to start with your family medical history today. What do you know about their medical history? 
Patient: My mother and father both had heart disease. Well, my mother had complication from her heart disease and that is how she passed.  My father was only in his forty's when he died. 
Doctor: I am so sorry the hear that. 
Patient: Thank you. I have two brothers, one whom I don't speak to very much anymore. I don't know if he has any health problems. My other brother is healthy with no issues. Both my uncles on my mother's side had polio, I think. 
Doctor: Tell me more about your uncles with polio. They both had polio? 
Patient: One of them had to wear crutches due to how bad his leg deformans were and then the other had leg deformities in only one leg.  I am fairly certain that they had polio.  
Doctor: Do you know of any other family

In [9]:
t2t = pipeline("text2text-generation", model="har1/HealthScribe-Clinical_Note_Generator", device=device)
# t2t = pipeline('text2text-generation', model='facebook/bart-base', device=device)

In [10]:
summary = t2t(sample[seq1id])

C:\ProgramData\anaconda3\envs\env_dl_cv_transformer_sentpiece\Lib\site-packages\transformers\models\bart\modeling_bart.py:597: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [11]:
print('Ground truth:\n')
print(sample[seq2id] + '\n\n')
print('Prediction:\n')
print(summary[0]['generated_text'])

Ground truth:

Symptoms: N/A.
Diagnosis: N/A.
History of Patient: Mother died of complications from heart disease, father died of heart disease in his 40s. Two living brothers, one with unknown medical history, the other apparently healthy. One healthy child. Maternal uncles had polio, one wore crutches due to severe leg deformans, the other had leg deformities in one leg. No other family members with neurological conditions known.
Plan of Action: N/A.



Prediction:

Symptoms: N/A
Diagnosis: None
History of Patient: Mother died of complications from heart disease, father died of heart disease in his 40s. Two brothers, one with unknown medical history, the other apparently healthy. Both of the patient's uncles had polio, one wore crutches due to severe leg deformans and the other had leg deformities in one leg. No other family members with neurological conditions known. Patient has one child who is healthy and well.
Plan of Action: Further evaluation and management of family medical hi

In [9]:
# Manual pipeline

model_name = 'facebook/bart-large-cnn'
model_name = 'facebook/bart-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenized_text = tokenizer(sample[seq1id], text_target=sample[seq2id], return_tensors='pt')
output = model.generate(**tokenized_text, num_beams=4, max_length=150, early_stopping=True)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

# del tokenizer, model

C:\ProgramData\anaconda3\envs\env_dl_cv_transformer_sentpiece\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Doctor: Hi there, sir! How are you today?   Patient: Hello! I am good.  Doctor: I would like to start with your family medical history today. What do you know about their medical history?  Patient: My mother and father both had heart disease. Well, my mother had complication from her heart disease and that is how she passed.  My father was only in his forty's when he died. Patience: I am so sorry the hear that.  PatienceDoctor: Thank you. I have two brothers, one whom I don't speak to very much anymore. I don’t know if he has any health problems. Both my uncles on my


In [18]:
tokenized_text

{'input_ids': tensor([[    0, 41152,    35,  2647,     6,    38,    33,   110,   381,   229,
           272,   266,     6,   924,    47,    33, 10272,   687,   326, 35600,
          6940,   493,     4,    96,    97,  1617,     6,   110,  1144,    16,
          4108,  3845,    87,  2340,   528,     7,  6379,  5834,     9, 10272,
           687, 37908,     4, 50118, 18276,  4843,    35,  8487,     4, 50118,
         41152,    35,  2647,     6,    89,    32,   117,   208,   255,  1022,
             4, 50118, 18276,  4843,    35,   289,   119,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[    0, 46994, 47629,    35,   234,    73,   250, 50118, 29038, 11244,
         13310,    35,  8356,   687,   326, 35600,  6940,   493, 50118, 38261,
             9

In [10]:
forwardpass = model.forward(**tokenized_text)

In [11]:
forwardpass

Seq2SeqLMOutput(loss=tensor(4.7388, grad_fn=<NllLossBackward0>), logits=tensor([[[32.4903,  5.4285, 16.5220,  ...,  5.8563,  5.6850,  1.4832],
         [10.1440, -0.9789, 10.6696,  ...,  0.5739,  1.4313,  0.2438],
         [-2.1545, -2.3693,  4.0702,  ..., -1.8813, -1.5010, -4.2861],
         ...,
         [-3.6289, -1.8304,  6.6317,  ..., -0.8985, -0.5385, -3.1141],
         [-2.0382, -1.6611,  8.4859,  ..., -0.5911, -0.4674, -2.8231],
         [-2.2806, -2.8756,  7.9112,  ..., -2.1137, -1.9660, -2.1610]]],
       grad_fn=<AddBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[-0.0350,  0.0088, -0.0005,  ...,  0.0112, -0.0014, -0.0085],
         [ 0.3622,  0.0527, -0.1266,  ..., -0.4243, -0.0873,  0.1291],
         [ 0.0840, -0.0956, -0.2983,  ..., -0.2193,  0.3024, -0.2208],
         ...,
         [-0.0980, -0.1325,  0.1586,  ..., -0.0278, -0.2407, -0.3338],
         [-0.0143, -0.0198,  0.2

In [15]:
forwardpass.logits.shape

torch.Size([1, 112, 50265])